In [ ]:
# 删除无效数据，去重，更改部分列名称

import pandas as pd
data = pd.read_csv("data.csv", encoding="gbk")
data.rename(columns={'1、姓名（不是 ID）': '姓名'}, inplace=True)
data.rename(columns={'2、ID': 'ID'}, inplace=True)
data.rename(columns={'3、手机号': '手机号'}, inplace=True)
data = data[data['姓名'] != 'aaa']
data = data.drop_duplicates(subset=['姓名', 'ID', '手机号'], keep='last', ignore_index=True)
# print(data)

In [ ]:
# 检查是否有疑似相同的问卷

# print(data)
for i in range(len(data)):
	for j in range(i):
		if data.loc[j, '姓名'] == data.loc[i, '姓名']\
		or data.loc[j, 'ID'] == data.loc[i, 'ID']\
		or data.loc[j, '手机号'] == data.loc[i, '手机号']:
			print(f'Warning: Row {j} and Row {i} may be the same person.')
			print(data.loc[j, ['姓名', 'ID', '手机号']])
			print(data.loc[i, ['姓名', 'ID', '手机号']])


In [ ]:
# 将列名称从款式补全为完整商品

cols = data.columns.tolist()
# print(cols)
phone_visited = False
for i in range(0,len(cols)):
	if cols[i] == '手机号':
		phone_visited = True
		continue
	if not phone_visited:
		continue
	if cols[i].split('、')[0] == cols[i-1].split('、')[0]:
		cols[i] = '('.join(cols[i-1].split('(')[:-1])+'('+cols[i].split('(')[-1]
# print(cols)
data.columns = cols
# print(data.columns)

In [ ]:
# 特殊处理

if data.loc[64, '姓名'] == '丁田' and data.loc[64, 'ID'] == '雾屿':
	data.drop(64, axis=0, inplace=True)
	data = data.reset_index(drop=True)

data.loc[data['ID'] == '菜菜', '17、宽大座垫(黑蓝)'] = 0
# print(data.loc[data['ID'] == '菜菜', '17、宽大座垫(黑蓝)'])
data['4、头巾(30)'] = 0

data.to_csv('output.csv', index=False, sep=',', encoding='gbk')

In [ ]:
# 对列统计

col_sums = data.sum(numeric_only=True, skipna=True).astype(int)

col_sums.to_csv('total.csv', index=True, sep=',', encoding='gbk')

In [ ]:
prices = pd.read_csv("prices.csv", encoding="utf-8")

# print(prices["商品"])
cols = data.columns.tolist()

sum = 0

charge_table = pd.DataFrame(columns=['姓名', '金额'])

for i in range(len(data)):
	total_100 = 0
	for col in cols:
		cnt = data.loc[i, col]
		if (prices['商品'] == col).any():
			if not pd.isna(cnt):
				unit_price = prices.loc[prices['商品'] == col, '单价'].iloc[0]
				total_100 += round(unit_price * 100) * int(cnt)
	sum += total_100
	charge_table.loc[len(charge_table)] = [data.loc[i, '姓名'], total_100 / 100]

# print(sum / 100)

charge_table.to_csv('charge.csv', index=True, sep=',', encoding='gbk')